# Visualization Project

---
**Authors**:
-  *Juan P. Zaldivar E.*
-  *Enrique Millán X.*
---

## Introduction

In this project, the focus is on analyzing collision data in New York City during the summers of 2018 and 2020. The primary objective is to develop a comprehensive static visualization that can address several key questions regarding the nature and patterns of these collisions. With the use of datasets related to collisions, weather conditions, and the New York City map, we aim to explore various facets, including the frequency of accidents, vehicle types involved, time of day occurrences, geographical hotspots, and potential correlations with weather conditions.

This file contains all the steps required to ensuring reproducibility of steps leading from raw data to a clean dataset is essential. The project is divided in three parts: the first part corresponds to the preprocessing of the data, the second part corresponds to the visualization desing process and the third part corresponds to the implementation of the visualization in the streamlit app to answer the questions.

The datasets are as follows:

- Collision Dataset: Extracting and filtering collision data specifically from June to September of 2018 and 2020. This involves selecting relevant columns, handling missing or inconsistent data, and ensuring data quality.

- Weather Dataset: Locating and incorporating weather data corresponding to the time frames and areas of interest.
- New York City Map: Acquiring a suitable map of New York City to overlay geographical information related to collision locations.

### Dataset obtention and description

The [*Collisions*](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95) dataset was already given by the instructors of the project. The Motor Vehicle Collisions crash table contains details on the crash event. Each row represents a crash event. The Motor Vehicle Collisions data tables contain information from all police reported motor vehicle collisions in NYC.

The *weather* dataset was obtained following the next steps:

- Visit the [NOAA Climate Data Online Search](https://www.ncdc.noaa.gov/cdo-web/search) web page.

- Select the following options:
  - `Weather Observation Type/Dataset -> Daily Summaries, Date Range -> 2018-01-01 to 2020-12-31, Search For -> Cities, Search Term -> New York City.`

- Look for "*New York, NY US*" and click in ADD TO CART. Now, click the cart in the top right corner.

- Select "*Custom GHCN-Daily CSV*", and the date previously selected (2018-01-01 to 2020-12-31). We are selecting more information than needed (to avoid disjoint downloads), but we will later filter it with ``Pandas`` and ``Open Refine``. Click continue.

- Fill the three options, and select "*metric units*".

- Fill all the options remaining and click continue. There are some options that will be probably not needed, but we will further analyze this when cleaning the datasets.

- Type the email where you want to receive the data so the order can start.


The *map* dataset was obtained from this [cartography web page](https://cartographyvectors.com/map/508-new-york-city-boroughs-ny).


The datasets are located in the folder `Data/` and the results are saved in the folder `Data/Preprocessed/`. Following are the loading of each dataset and the import of the required libraries.

### Libraries

In [ ]:
pip install altair==5.1.2 pandas==1.5.3 numpy==1.23.5 altair==5.1.2 h3pandas==0.2.5 geopandas==0.13.2 vegafusion[embed]>=1.4.0

In [ ]:
import os
import numpy as np
import pandas as pd
import altair as alt
import h3pandas as h3
import geopandas as gpd
from shapely.geometry import Point
from Modules import preprocessing as pre

## Dataset preprocessing

The preprocessing of the files involved a collaborative effort using both OpenRefine and selected Python libraries. This strategic approach was adopted to take advantage of the unique strengths and capabilities offered by each tool. OpenRefine facilitated initial data cleaning and transformation tasks, with its user-friendly interface for effective manipulation of datasets. Simultaneously, Python libraries were utilized to perform more complex data operations and manipulations, with special emphasis on the extensive functionalities and flexibility they provide. This combination allowed for a comprehensive preprocessing workflow that maximized efficiency and accuracy in preparing the data for subsequent analyses and visualization tasks.

In [ ]:
dir = './Data'
collision_exists = False

## Collision dataset preprocessing

The initial step involved loading the original dataset into a `Pandas` dataframe, primarily to apply a date range filter efficiently. The rationale behind this approach was to optimize the data filtering process, considering the considerable size of the original dataset. The volume of data posed challenges within OpenRefine, leading to slow and inefficient computational processes. By filtering the dataset using `Pandas`, it allowed for a more streamlined and quicker selection of the desired date range. Following this initial filtering phase, the refined dataset was exported as a `.csv` file and subsequently imported into OpenRefine for further data processing and cleaning procedures. This sequential approach ensured a balance between computational efficiency and data handling capabilities across both `Pandas` and OpenRefine, resulting in a more effective preprocessing workflow.

In [ ]:
if os.path.exists(f'{dir}/collisions_2018-2020.csv'):
    collision = pd.read_csv(f'{dir}/collisions_2018-2020.csv')
    colission_exists = True
else:
    try:
        collision = pd.read_csv(f'{dir}/collisions.csv')
        collision.shape
    except:
        print('Raw data missing, please upload the data before continuing')

In [ ]:
collision.head()

If the filtered version does not exist, we proceed with the filtering.

In [ ]:
if not colission_exists:
    collision = pre.time_filter(collision, 'CRASH DATE')
    collision.to_csv(f'{dir}/collisions_2018-2020.csv', index=False)

collision.shape

Following the initial filtering process, the refined dataset was exported as a `.csv` file and then imported into OpenRefine. The approach taken in OpenRefine for data manipulation and cleaning will be thoroughly explained, providing reasoning and justifications for the specific procedures employed. This section aims to outline the methods used in OpenRefine to ensure transparency and clarity regarding the data processing steps, offering insights into why certain actions were taken to achieve the desired clean and structured dataset.

### Data type conversion

In OpenRefine, the data conversion process involved several attribute adjustments. The **CRASH DATE** attribute underwent a conversion to a date type for enhanced consistency and data clarity. Meanwhile, both **COLLISION ID** and **CRASH TIME** were temporarily set as string types.

Attributes pertaining to the geographical location of the collisions were modified to strings, accompanied by specific notations. As part of this process, all values were standardized to uppercase, and any extra spaces were removed where applicable. This standardization was implemented to facilitate the effectiveness of the clustering method utilized for collectively inspecting and modifying cells. The objective was to streamline the identification and correction of any inconsistencies or inaccuracies within the data, ensuring a more uniform and reliable dataset for subsequent analyses.

The attributes pertaining to the number of persons involved in the collision underwent a data type conversion to integers within the dataset. This decision was driven by the discrete nature of these values and the expectation that these numerical counts wouldn't contain negative values.

Conversely, the attributes related to vehicles and factors involved in the collisions were retained as strings temporarily. This choice was made to maintain flexibility in handling these attributes during subsequent data processing and analysis phases, ensuring that any necessary modifications or categorizations could be applied effectively as the analysis progressed.

### Data selection and transformation

All of the following transformations were applied with OpenRefine. However, certain validations and checks to substantiate and validate these transformations are conducted within this specific section of the notebook.

In [ ]:
try:
    precollision = pd.read_csv(f'{dir}/collisions_2018-2020_prepro_v1.csv')
except:
    print('Preprocessed data from Openrefine missing, please upload the data before continuing')

#### Time attributes

Convertion of **CRASH DATE** column in the precollision DataFrame into a pandas datetime format, allowing for easier handling and manipulation of date-related operations. The next operations and column were made:

- Creation of a new column named **YEAR** in the precollision DataFrame.

- Creation of a new column named **DAY NAME** in the precollision DataFrame. This operation helps categorize each entry by the specific day of the week on which the collision occurred.

- Creation of a new column called **TYPE OF DAY** in the precollision DataFrame, it assigns the corresponding row in the "TYPE OF DAY" column as "Weekend". Otherwise, it assigns "Weekday".

In [ ]:
precollision["CRASH DATE"] = pd.to_datetime(precollision["CRASH DATE"])
precollision['YEAR'] = precollision['CRASH DATE'].astype(str).str[:4]
precollision["DAY NAME"] = precollision["CRASH DATE"].dt.day_name()
precollision["TYPE OF DAY"] = np.where(precollision["DAY NAME"].isin(["Saturday", "Sunday"]), "Weekend", "Weekday")

for _ in range(3):
    cols = precollision.columns.tolist()
    cols = cols[:1] + cols[-1:] + cols[1:-1]
    precollision = precollision[cols]

precollision.head(1)

Additionally, a new column named **CRASH TIME INTERVAL** is created in the precollision DataFrame, that formats **CRASH TIME** as a two-digit string representation to isolate the hour.

In [ ]:
precollision['CRASH TIME'] = pd.to_datetime(precollision['CRASH TIME'], format='%H:%M').dt.time
precollision['CRASH TIME INTERVAL'] = precollision['CRASH TIME'].apply(lambda x: f"{x.hour:02}")
precollision.drop(columns=['CRASH TIME'], inplace=True)

precollision['CRASH MOMENT'] = precollision['CRASH TIME INTERVAL'].apply(pre.categorize_moment)

# move TIME INTERVAL to the fourth column
cols = precollision.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
precollision = precollision[cols]

# move CRASH MOMENT to the fifth column
cols = precollision.columns.tolist()
cols = cols[:4] + cols[-1:] + cols[4:-1]
precollision = precollision[cols]

In [ ]:
precollision.head(1)

#### Geographical attributes

At first glance, **ON STREET NAME** and **OFF STREET NAME** seem to be the same attribute, but with different names. The web site of the dataset cointains the following descriptions:

- **ON STREET NAME**: *Street on which the collision occurred*.
- **OFF STREET NAME**: *Street address if known*.

This gives the idea that both attributes probably contain the same information. Furthermore, there are no rows with both attributes filled, which makes the idea of merging both attributes plausible and would consolidate information without redundancy.

**Note**: *The utilization of the `collision` dataframe instead of the `precollision` dataframe is due to the distinction in their contents and alterations within the OpenRefine interface. Specifically, the `collision` DataFrame account for the actions carried out in the OpenRefine interface, illustrating the sequence of transformations and modifications made to the dataset.*

*In contrast, the `precollision` DataFrame encapsulates the actual changes implemented within OpenRefine. This differentiation is crucial to clarify that the `collision` DataFrame primarily functions as an illustrative tool for explicating certain actions executed within the OpenRefine environment.*

*Therefore, `collision` is used solely for explanatory purposes, offering insight into the sequence of actions performed during the data manipulation process within OpenRefine, while the actual modifications and alterations reside within the `precollision` DataFrame.*

In [ ]:
collision[(collision['ON STREET NAME'].notnull()) & (collision['OFF STREET NAME'].notnull())].shape

In [ ]:
collision[(collision['ON STREET NAME'].notnull()) | (collision['OFF STREET NAME'].notnull())].shape

The resulting attribute after merging both columns is called **STREET NAME** and contains the street name/address where the collision occurred, with no missing values. Some rows will have a more detailed description of the street, while others will only have the name of the street.

**CROSS STREET**, which is the third attribute related to the street enviroment, and represents the name of the closest street crossing the street of the crash. Upon analysis, it has been determined that this attribute does not hold relevance or utility for the intended analysis objectives. Therefore, in the interest of streamlining the dataset and focusing on pertinent factors, the CROSS STREET attribute is considered non-essential and will be dropped from the dataset.

Similarly, **LOCATION** seems to contain the tuple (**LATITUDE**, **LONGITUDE**), so we could, a priori, remove the two extra attributes.

In [ ]:
collision[(collision['LOCATION'].notnull()) & (collision['LATITUDE'].notnull()) & (collision['LONGITUDE'].notnull())].shape

The number of rows where the three attributes are not missing does not cover the total number of rows, but there are no rows where the **LOCATION** attribute is missing and at least one of the other two attributes is not missing.

In [ ]:
collision[(collision['LOCATION'].isnull()) & (collision['LATITUDE'].isnull()) & (collision['LONGITUDE'].notnull()) | (collision['LATITUDE'].notnull()) & (collision['LONGITUDE'].isnull())].shape

In [ ]:
collision[(collision['LOCATION'].isnull())].shape

Which makes the rest of the rows (7667) with missing values in the three attributes. This means that the **LATITUDE** and **LONGITUDE** attributes can be removed, since the **LOCATION** attribute contains the same information. With this transformation, the number of attributes is reduced by two.

The clustering process utilized the key collision method in conjunction with the fingerprint keying function, applied separately to each individual geographical attribute. After a couple of iterations, no substantial alterations in attribute values were identified. However, during this process, misspellings were detected and rectified to ensure data accuracy.

The misspellings were corrected and the clusterization was done again. The results were the same, which means that the values were already consistent. To verify the result, a Neares Neighbours analysis was done as well but without finding any significant variation.

#### Vehicle attributes

Regarding vehicle information, the statement of the project specifies that only the **VEHICLE CODE TYPE 1** is of interest to the visualization, leaving the remaining vehicle codes (2-5) irrelevant. Consequently, the associated contributing factors linked to these other vehicle types can also be excluded from the analysis.

We have seen already that there are many classes of the **VEHICLE CODE TYPE 1** values. To simplify this complexity, we opted to reduce the diversity of classes by employing a clustering technique. This clustering methodology allowed us to condense the multitude of classes within **VEHICLE CODE TYPE 1** into a more manageable set of clusters, facilitating a more comprehensible and concise representation for subsequent visualization and analysis purposes.

In [ ]:
len(collision['VEHICLE TYPE CODE 1'].unique())

With the clusterization (key collision and fingerprint keying function) of the **VEHICLE TYPE CODE 1** attribute we found a lot of misspellings and inconsistencies. The clusterization was done iteratively, correcting the misspellings and inconsistencies found in each iteration.

After several iterative steps, wherein numerous subclasses were consolidated into similar types of vehicles, resulting in a more aggregated dataset, the count of distinct classes decreased significantly. This reduction allowed for the remaining subclasses to be manually merged together, given their similarity or shared characteristics.

For this manual clusterization, the `clusterize_vehicle_type` function was defined that aims to categorize or group different types of vehicles within a given DataFrame based on specific criteria.

In [ ]:
precollision = pre.clusterize_vehicle_type(precollision, 'VEHICLE TYPE CODE 1')

For example, certain types like 'SUV', 'FLAT', '3-DOOR', etc., are replaced with a common category name 'CAR'. Similarly, other types like 'BULK AGRICULTURE', 'PK', 'TANK', etc., are grouped under 'OTHERS'.

The resulting classes of the **VEHICLE CODE TYPE 1** attribute are:

In [ ]:
len(precollision['VEHICLE TYPE CODE 1'].unique()), precollision['VEHICLE TYPE CODE 1'].unique()

The chosen vehicle types were selected deliberately to strike a balance between having an excessive number of classes and too few to provide meaningful insights in the analysis. This selection aimed to offer the user a manageable yet comprehensive view of vehicle categories, ensuring that the analysis remains insightful without overwhelming the dataset with excessive vehicle classifications.

In [ ]:
precollision['VEHICLE TYPE CODE 1'].value_counts()

Similar strategy was done with the **CONTRIBUTING FACTOR VEHICLE 1** attribute. However, the aggregation was not so exhaustive since this attribute wasn't needed a priori for the main questions that the visualizations should answer. For this attribute basic merge transformations were applied in OpenRefine until no "strange" or "uninformative" nor repeated classes remained.

In [ ]:
len(precollision['CONTRIBUTING FACTOR VEHICLE 1'].unique()), precollision['CONTRIBUTING FACTOR VEHICLE 1'].unique()

#### Number of persons attributes

For the visualization purposes, the differentantion of **PERSONS**, **PEDESTRIANS**, **CYCLISTS** and **MOTORISTS** (**INJURED/KILLED**) is irrelevant. A more useful attribute would be the total number of persons involved in the collision. This can be obtained by summing the four attributes under the assumption that the **PERSONS** attribute is not the sum of the other three attributes.

This condition was needed to be checked because the documentation of the dataset was not precise enough to determinate if **NUMBER OF PERSON INJURED/KILLED** was an aggregate from the other three columns or not.

*Note: The metadata information available in the web of the dataset was: "Number of persons injured/killed" regarding the **NUMBER OF PERSONS INJURED/KILLED**.*

In [ ]:
collision['NUMBER OF PERSONS INJURED'].equals(collision['NUMBER OF PEDESTRIANS INJURED'] + collision['NUMBER OF CYCLIST INJURED'] + collision['NUMBER OF MOTORIST INJURED'])

In [ ]:
collision['NUMBER OF PERSONS INJURED'].equals(collision['NUMBER OF PEDESTRIANS INJURED'])

In [ ]:
collision['NUMBER OF PERSONS KILLED'].equals(collision['NUMBER OF PEDESTRIANS KILLED'] + collision['NUMBER OF CYCLIST KILLED'] + collision['NUMBER OF MOTORIST KILLED'])

In [ ]:
collision['NUMBER OF PERSONS KILLED'].equals(collision['NUMBER OF PEDESTRIANS KILLED'])

As seen by the logical comprobations, the **NUMBER OF PERSONS INJURED/KILLED** is not the sum of the other three attributes. Furthermore, the terms persons and pedestrians are not equal, as one could have thought that the term persons was used to refer to pedestrians.

Based on this, the discrete attributes refering to the injured people were summed to obtain **NUMBER OF INJURED** and the discrete attributes refering to the killed people were summed to obtain **NUMBER OF KILLED**. The **NUMBER OF INJURED/KILLED** attributes were removed.

#### OpenRefine results

In [ ]:
precollision.head(1)

At this point, the dataset contains the attributes needed (with the *weather* attributes as an exception) for the analysis and some extra attributes that were considered interesting for some possible extra analysis or insights that we could think about.

### Missing values

It has already been mentioned the existence of some missing values. In the previous section, the verification of missing values was done with the ``.isnull()`` method of ``Pandas``. However, this method does not take into account the ``NaN`` values. In order to check the existence of ``NaN`` values, the ``.isna()`` method was used.

In [ ]:
comp = (precollision.isnull().sum() == precollision.isna().sum())
comp[comp == False]

As seen previously, all the missing values of the dataset are detected both with ``.isnull()`` and ``.isna()``. After this check, we could group the attributes with missign values in three separeted clusters:
- Geographical attributes
- Injured/Killed attributes
- Vehicle attributes

In [ ]:
precollision.isnull().sum()

#### Imputation of geographic attributes

The first cluster is formed with reference to the geographicals attributes. The attributes in this cluster are:
- **BOROUGH**
- **ZIP CODE**
- **LOCATION**
- **STREET NAME**
- **CROSS STREET NAME**

Notice that the attributes with the less missing values is **STREET NAME** with only a $0.20\%$ of the entire dataset, partially thanks to the merge of **ON STREET** and **OFF STREET** attributes in the previous sections.

In [ ]:
precollision['LOCATION'].isnull().sum(), precollision['STREET NAME'].isnull().sum()

As the usage of the **STREET NAME** attribute remains uncertain for analysis, we have opted to retain the missing values within this attribute. Instead, our focus will be on removing rows with missing values in the **LOCATION** attribute. This decision is motivated by the small proportion of missing values in the **LOCATION** attribute relative to the entire dataset of `precollision`. Specifically, the missing values in **LOCATION** account for only $6.6\%$ of the total rows within the dataset. This strategy aims to ensure data completeness in crucial fields while acknowledging the relatively minor impact of missing **LOCATION** data compared to the entire dataset.

Based on reviews, it has come to our attention that merging the **LONGITUDE** and **LATITUDE** columns into a singular **LOCATION** attribute was an error. Several operations require distinct handling of each geometric attribute independently. Consequently, we have reversed this consolidation by dividing the **LOCATION** attribute back into its original two separate attributes: **LONGITUDE** and **LATITUDE**. This restoration will allow us to conduct specific operations and analyses on each geographical coordinate individually, ensuring accuracy and facilitating targeted manipulations or computations as needed.

In [ ]:
precollision[['LATITUDE', 'LONGITUDE']] = precollision['LOCATION'].str.split(', ', expand=True)
precollision['LATITUDE'] = precollision['LATITUDE'].str.replace('(', '')
precollision['LONGITUDE'] = precollision['LONGITUDE'].str.replace(')', '')

precollision['LATITUDE'] = precollision['LATITUDE'].astype(float)
precollision['LONGITUDE'] = precollision['LONGITUDE'].astype(float)

precollision.drop(columns=['LOCATION'], inplace=True)

A simpler method for value imputation, as an alternative to directly imputing from the **STREET NAME** attribute, involves checking if a point specified by its coordinates (**LONGITUDE**, **LATITUDE**) falls within the boundary polygon of different boroughs. If the original attribute value is null, this method aims to assign the specific borough corresponding to the geographic location of the provided coordinates.

In [ ]:
nyc_map = gpd.read_file('Data/new-york-city-boroughs-ny_.geojson')

boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

# Extraction of the borough polygon
borough_poly = {}
for b in boroughs:
    poly = nyc_map[nyc_map['name'] == b]['geometry']
    borough_poly[b] = poly.values[0]

In [ ]:
if not os.path.exists(f'{dir}/precollision_2018-2020_prepro_v2.csv'):
    for idx, row in precollision.iterrows():
        if row['BOROUGH'] is not None:
            lon = row['LONGITUDE']
            lat = row['LATITUDE']

            if lat is not None and lon is not None:
                p = Point(lon, lat)
                for b, poly in borough_poly.items():
                    if p.within(poly):
                        precollision.loc[idx, 'BOROUGH'] = b.upper()
                        break

    precollision.to_csv(f'{dir}/precollision_2018-2020_prepro_v2.csv', index=False)
else:
    precollision = pd.read_csv(f'{dir}/precollision_2018-2020_prepro_v2.csv')

In [ ]:
precollision['LATITUDE'].isna().sum(), precollision['BOROUGH'].isna().sum()

We observe a considerably lower number of missing values in the **BOROUGH** attribute compared to the missing values in the coordinates.

#### Imputation of vehicle attributes

In some rows of the dataset, the **CONTRIBUTING FACTOR VEHICLE** is missing but the **VEHICLE TYPE CODE** is not. This suggests that the vehicle type is known, but the factor that contributed to the collision is not. In order to fill this missing values, the factor was set as *unespecified*. This was done for all the rows and columns where the **CONTRIBUTING FACTOR VEHICLE** was missing with the above condition.

In [ ]:
pre.imputation_with_ref_col(precollision, 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'Unspecified')

Likewise, in some rows of the dataset, the **VEHICLE TYPE CODE** is missing but the **CONTRIBUTING FACTOR VEHICLE** is not. This suggests that the factor that contributed to the collision is known, but the vehicle type is not. In order to fill this missing values, the vehicle type was set as *unknown*. This was done for all the rows and columns where the **VEHICLE TYPE CODE** was missing with the above condition.

In [ ]:
pre.imputation_with_ref_col(precollision, 'VEHICLE TYPE CODE 1', 'CONTRIBUTING FACTOR VEHICLE 1', 'UNKNOWN')

In [ ]:
precollision.isnull().sum()

#### Imputation of number of person attributes

In [ ]:
precollision[precollision['NUMBER OF INJURED'].isnull() | precollision['NUMBER OF KILLED'].isnull()]

Since the proportion of resulting rows with missing values in the count of persons involved in the collision is relatively minor compared to the entire dataset, the decision was made to impute these missing values by setting them to 0. This assumption is based on the premise that in these cases, no individuals were involved in the collision events where the data for this attribute was absent.

Given the small percentage of the rows $0.01\%$, it was considered that there would not be a significant impact in the final visualization whether the missing values were set to 0 or the rows with missing values were dropped. However, we decided to set the missing values to 0 in order to keep the rows and not lose some of its information. This was done also because **NUMBER OF INJURED** and  **NUMBER OF KILLED** were attributes that are not necessary for the main visualizations and we were only keeping them in case we found an intereseting extra visualization with them.

In [ ]:
precollision['NUMBER OF INJURED'].fillna(0, inplace=True)
precollision['NUMBER OF KILLED'].fillna(0, inplace=True)

In [ ]:
precollision.isnull().sum()

Considering the extent of missing data in the columns mentioned above and the fact that the **ZIP CODE** and **STREET NAME** columns are deemed irrelevant for the analysis, it may be reasonable to contemplate deleting rows with missing values. However, the decision to remove rows with missing values should be made cautiously, taking into account the impact on the overall analysis and the importance of the information contained in each column.

Since the missing values in **BOROUGH**, **LATITUDE**, and **LONGITUDE** columns are relatively small in comparison to the total dataset size and the analysis does not heavily rely on these specific columns, removing rows with missing values could be a viable option. Nonetheless, it's crucial to assess the potential loss of valuable information and the significance of these columns in the context of the analysis before proceeding with the deletion of rows.

In [ ]:
precollision[(precollision['BOROUGH'].isnull()) | (precollision['LONGITUDE'].isnull())].shape

In [ ]:
8820*100/115740

### Save the results

In [ ]:
precollision.to_csv(f'{dir}/collisions_clean.csv', index=False)

In [ ]:
precollision.columns

## Weather dataset preprocessing

During the initial data download phase, all available attributes were selected. Subsequently, we intend to dive into the dataset to identify and select specific attributes that align with our project objectives. Our aim is to explore the entirety of the dataset comprehensively, evaluating each attribute's relevance and usefulness towards fulfilling our intended goals. This exploration will involve careful consideration of the attributes' significance in addressing our project's specific inquiries. Attributes consoidered pertinent and conducive to addressing our objectives will be retained for further analysis, while those less relevant or unrelated will be excluded from subsequent stages of our analysis and visualization process.

In [ ]:
weather = pd.read_csv(f'{dir}/weather.csv')

### Data exploration

In [ ]:
weather.shape

Since we are only interested in the rows where the date is inside the timeranges of 01/06/2018 - 31/09/2018 and 01/06/2020 - 31/09/2020, we will filter the data to only include those rows.

In [ ]:
if os.path.exists(f'{dir}/weather_2018-2020.csv'):
    weather = pd.read_csv(f'{dir}/weather_2018-2020.csv')
else:
    weather = pre.time_filter(weather, 'DATE')
    weather.to_csv(f'{dir}/weather_2018-2020.csv', index=False)

weather.shape

In [ ]:
weather.columns

By looking at the documentation of the weather datatset and the attributes present, each row represents some selected observations (values) available for a given **STATION** and **DATE**. Neither the **STATION** nor the **DATE** are unique.

In [ ]:
len(weather['STATION'].unique()), len(weather['DATE'].unique())

Apart from the first 6 columns (**SATTION**, **NAME**, **LATITUDE**, **LONGITUDE**, **ELEVATION** and **DATE**), the rest of the attributes correspond to optional flags and their respective attributes (definded by the weather documentation *Note: The 4 flags listed [...] are optional on the Custom GHCN-Daily ASCII Form*) and therefore can contain several null values. That is the reason for the large quantity of sparse attributes detected. We will explore the data to see which attributes are useful for our purpose.

All these atributes correspond to the Table 4. A brief description is collected in the following table:

| Attribute | Description |
| --- | --- |
| *PRCP* | Precipitation (mm) |
| *SNOW* | Snowfall (mm) |
| *SNWD* | Snow depth (mm) |
| *TSUN* | Daily total sunshine (minutes) |
| *TMAX* | Maximum temperature (Celsius) |
| *TMIN* | Minimum temperature (Celsius) |
| *TAVG* | Average temperature (Celsius) |
| *TOBS* | Temperature at the time of observation |
| *AWND* | Average daily wind speed (meters per second) |
| *WT** | Weather type * |


The next table contains the attributes that we considered that are not useful for the visualization purpose:

| Attribute | Description |
| --- | --- |
| *PGTM* | Peak gust time (hours and minutes, i.e., HHMM) |
| *DARP* | Number of days included in the multiday precipitation total (MDPR) |
| *DASF* | Number of days included in the multiday snowfall total (MDSF) |
| *MDPR* | Multiday precipitation total (mm; use with DAPR and DWPR, if available) |
| *MDSF* | Multiday snowfall total (mm; use with DASF and DWSF, if available) |
| *PSUN* | Daily percent of possible sunshine (percent; use with TSUN, if available) |
| *WSF** | Fastest *-minute wind speed (meters per second) |
| *WDF** | Direction of fastest *-second wind (degrees) |
| *WESD* | Water equivalent of snow on the ground (decimal mm) |
| *WESF* | Water equivalent of snowfall (decimal mm) |


Both **DARP** and **DASF** don't directly measure specific weather conditions. They are more of an aggregate measure of precipitation/snowfall over multiple days, which is not useful for our purpose. The same applies to **MDPR**, **MDSF** and **PSUN** that are also not useful for our purpose since they are not specific weather conditions.
Other attributes such as **WDF**, **WSF**, **WESD**, **WESF** or **PGTM** (among others) give data that is diffuclt to collect, therefore having a lot of null values, and is too specific to be useful for our analysis. For instance, all wind related attributes are not really useful for our analysis, and a simple and general attribute such as **AWND** is much more useful for our task.

The ***_ATTRIBUTES** columns refer to the rest of the tables in the documentation (Table 1, 2, 3). These tables serve as flags for the measurement, quality and source of the data respectively. The values available for these attributes cover a wide range. In the data selection process we will analyze the subset of those values that are present in the data and discuss if they are useful for our purpose.

### Data selection

#### Geographical attributes

We are planning to remove the **ELEVATION** column from our dataset. This column doesn't bear any apparent correlation or relevance to the existing dataset, and the information it provides isn't conducive to our visualization objectives. As such, eliminating this column will streamline the dataset for our specific analytical and visualization purposes.

In [ ]:
weather.drop(columns=['ELEVATION'], inplace=True)

Given the presence of the coordinates attributes, the necessity of utilizing the **STATION** code or **NAME** seems redundant or unnecessary.

In [ ]:
weather.drop(columns=['STATION', 'NAME'], inplace=True)

#### Observational attributes

Since we are analyzing the summer periods of 2018 and 2020, it is highly unlikely to have observations of snow in the records. Under this assumption we can drop the **SNOW** and **SNWD** attributes.

In [ ]:
weather['SNOW'].unique(), weather['SNWD'].unique()

As we supposed, the only observations registred of **SNOW** and **SNWD** are null values or 0. Therefore, we can drop these columns and their respective flags.

In [ ]:
weather.drop(columns=['SNOW', 'SNWD', 'SNOW_ATTRIBUTES', 'SNWD_ATTRIBUTES'], inplace=True)

From the data exploration done in the previous section, we can remove the following attributes and their respective flags since we concluded their utility is really limited for our purpose: **DARP**, **DASF**, **MDPR**, **MDSF**, **PSUN**, **WDF**, **WESD**, **WESF** .

In [ ]:
weather.drop(columns=['DAPR',
                        'DASF',
                        'PGTM',
                        'MDPR',
                        'MDSF',
                        'PSUN',
                        'WDF2',
                        'WDF5',
                        'WESD',
                        'WESF',
                        'WSF2',
                        'WSF5',
                        'DAPR_ATTRIBUTES',
                        'DASF_ATTRIBUTES',
                        'MDPR_ATTRIBUTES',
                        'MDSF_ATTRIBUTES',
                        'PSUN_ATTRIBUTES',
                        'WDF2_ATTRIBUTES',
                        'WDF5_ATTRIBUTES',
                        'WESD_ATTRIBUTES',
                        'WESF_ATTRIBUTES',
                        'WSF2_ATTRIBUTES',
                        'WSF5_ATTRIBUTES'
                        ], inplace=True)

weather.columns

Regarding the **WT*** columns, they all describe a specific weather condition that could be referred as *adverse condition* given the descriptions from the documentation.


| Attribute | Description |
| --- | --- |
| *WT02* | Heavy fog or heaving freezing fog (not always distinguished from fog) |
| *WT03* | Thunder |
| *WT04* | Ice pellets, sleet, snow pellets, or small hail" |
| *WT05* | Hail (may include small hail) |
| *WT06* | Glaze or rime |
| *WT08* | Smoke or haze |
| *WT09* | Blowing or drifting snow |
| *WT11* | High or damaging winds |

As checked, they all have the null values in the same rows, which could indicate that the ``nan`` value could mean a ``False`` value for all the adverse type of weather conditions, that is the corresponding record refers to a *normal* weather day. Therefore, we will replace the ``nan`` values with ``0``.

In [ ]:
weather[weather['WT01'].isnull() & weather['WT02'].isnull() & weather['WT03'].isnull() & weather['WT04'].isnull() & weather['WT05'].isnull() & weather['WT06'].isnull() & weather['WT08'].isnull() & weather['WT09'].isnull() & weather['WT11'].isnull()].shape

In [ ]:
weather['WT01'].unique(), weather['WT02'].unique(), weather['WT03'].unique(), weather['WT04'].unique(), weather['WT05'].unique(), weather['WT06'].unique(), weather['WT08'].unique(), weather['WT09'].unique(), weather['WT11'].unique()

With this knowledge, the final decision was to merge all the **WT*** columns into a single one called **ADVERSE CONDITION**, which would have boolean values (0 or 1) indicating if the day was normal or not.

In [ ]:
wt_columns = ['WT01', 'WT02', 'WT03', 'WT04', 'WT05', 'WT06', 'WT08', 'WT09', 'WT11']

weather['ADVERSE_CONDITIONS'] = weather[wt_columns].isnull().all(axis=1).astype(int)
weather['ADVERSE_CONDITIONS'] = 1 - weather['ADVERSE_CONDITIONS']

weather['ADVERSE_CONDITIONS'].value_counts()

The flag attributes of the **WT*** columns will be removed given that the weather conditions columns were aggregated into a single column and thus the flags are not meaningful anymore.

In [ ]:
wt_columns_att = [col + '_ATTRIBUTES' for col in wt_columns]

weather.drop(columns=wt_columns, inplace=True)
weather.drop(columns=wt_columns_att, inplace=True)

weather.columns

#### Flags attributes

As done in the previous section, the ***_ATTRIBUTES** contain specific and irrelevant information  for our task, so it was decided to remove them all.

In [ ]:
cols = weather.columns.tolist()

for col in cols:
    if '_ATTRIBUTES' in col:
        print(col, weather[col].unique(), sep=':')

In [ ]:
wt_columns_att = [col for col in weather.columns if '_ATTRIBUTES' in col]

weather.drop(columns=wt_columns_att, inplace=True)

### Missing values

Considering the sparsity observed during data exploration, it is prudent to identify columns primarily composed of missing values for potential removal from the dataset.

In [ ]:
weather.shape

As can be seen, **TSUN** column contains only missing values. Therefore, we will remove the column.

In [ ]:
weather.isnull().sum()

In [ ]:
weather.drop(columns=['TSUN'], inplace=True)

Regarding the **PRCP** attribute, we'll proceed under the assumption that null values equate to zero. This assumption is based on the premise that the absence of recorded precipitation (signified by null values) suggests no actual precipitation occurred. We justify this assumption due to null values constituting approximately 10% of the dataset, which is a relatively moderate proportion. Furthermore, considering the dataset pertains to the summer season, a period typically characterized by lower precipitation occurrences, this assumption appears to carry a lesser risk as well.

In [ ]:
weather['PRCP'].fillna(0.0, inplace=True)

Regarding temperature, the great number of missing values is a problem. We will impute the missing values with the avergae temperature per day of each attribute (*TMAX*, *TMIN*, *TOBS*).

In [ ]:
weather['TMAX'] = weather.groupby('DATE')['TMAX'].transform(lambda x: x.fillna(x.mean()))
weather['TMIN'] = weather.groupby('DATE')['TMIN'].transform(lambda x: x.fillna(x.mean()))
weather['TOBS'] = weather.groupby('DATE')['TOBS'].transform(lambda x: x.fillna(x.mean()))

Since the three attributes from above have been imputed, and **TAVG** contains the most missing values, it was decided to remove this row. In case we need an average for temperature we can compute it with the previous attributes.

In [ ]:
weather.drop(columns=['TAVG'], inplace=True)

In [ ]:
weather.isnull().sum()

We will apply the same strategy on AWND to get rid of the missing values.

In [ ]:
weather['AWND'] = weather.groupby('DATE')['AWND'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
weather.isnull().sum()

For possible future analysis we will categorize the **AWND** and **PRCP** with the following scales:

- **AWND** with Beaufort scale

- **PRCP**  following the classification of the World Meteorological Organization, 2018(https://www.researchgate.net/figure/Rain-intensity-classifications-according-to-the-World-Meteorological-Organization-2018_tbl1_353769617)


### Data transformation

It has been recognized that addressing the final question regarding the correlation between weather conditions and accidents necessitates the aggregation of weather measurements on a daily basis. This makes the coordinate attributes irrelevant as they pertain to specific measurement locations.

However, rather than replacing the original dataset with an aggregated version, the decision has been made to generate a duplicate dataset named `weather_aggregated`. This duplicate dataset will serve as a copy where the necessary transformations and aggregations can be conducted while retaining the integrity of the original dataset for reference and comparative analysis.

In [ ]:
weather_aggregated = weather.copy()
weather_aggregated.drop(columns=['LONGITUDE', 'LATITUDE', 'ADVERSE_CONDITIONS'], inplace=True)

In [ ]:
weather_aggregated = weather_aggregated.groupby('DATE').mean().reset_index()
weather_aggregated.head()

Categorizing the Average Wind Speed (AWND) attribute with reference to the [Beaufort scale](https://en.wikipedia.org/wiki/Beaufort_scale) can provide several advantages for visualization and analysis purposes:

1. **Standardization:** Allows for a standardized representation of wind intensity. This categorization simplifies the interpretation of wind speed data, making it more intuitive and comprehensible.

2. **Visualization Clarity:** By mapping AWND values to the Beaufort scale categories, it facilitates clearer visualizations.

4. **Enhanced Interpretation:** The descriptive labels (e.g., calm, moderate breeze, strong gale), enable an easier interpretation for individuals less familiar with raw wind speed values.

In [ ]:
weather_aggregated['BEAUFORT_SCALE'] = weather_aggregated['AWND'].apply(pre.beaufort_scale)
weather_aggregated['BEAUFORT_SCALE'].unique()

As well, categorizing precipitation levels **PRCP** using this scale simplifies the interpretation of rain intensity patterns, aiding in visualizations by offering descriptive categories that represent different levels of precipitation intensity, from slight to violent. These categories can enhance the understanding of the relationship between precipitation levels and other variables, facilitating the exploration and analysis of weather impact on collision occurrences.

In [ ]:
weather_aggregated['PRCP_SCALE'] = weather_aggregated['PRCP'].apply(pre.rain_intensity_scale)
weather_aggregated['PRCP_SCALE'].unique()

In [ ]:
weather_aggregated['MEAN_TEMP'] = weather_aggregated[['TMAX', 'TMIN', 'TOBS']].mean(axis=1)

### Save the results

In [ ]:
weather.to_csv(f'{dir}/weather_clean.csv', index=False)
weather_aggregated.to_csv(f'{dir}/weather_aggregated.csv', index=False)

## Visualization creation process

In all visualizations we have chosen color palettes suitable for color blindness, following the [Colorblind Safe Color Schemes](https://www.nceas.ucsb.edu/sites/default/files/2022-06/Colorblind%20Safe%20Color%20Schemes.pdf).

In [ ]:
weather = pd.read_csv(f'{dir}/weather_clean.csv')
collisions = pd.read_csv(f'{dir}/collisions_clean.csv')
nyc_map = gpd.read_file(f'{dir}/new-york-city-boroughs-ny_.geojson')
weather_aggregated = pd.read_csv(f'{dir}/weather_aggregated.csv')


collisions['YEAR'] = collisions['YEAR'].astype(str)

alt.data_transformers.enable("default")

colores_hex = [
    '#a3ffd6',  # Verde agua intenso
    '#d69bf5',  # Púrpura intenso
    '#ff8080',  # Rojo intenso
    '#80ff80',  # Verde intenso
    '#80bfff',  # Azul intenso
    '#ffff66',  # Amarillo intenso
    '#ffcc66',  # Naranja intenso
    '#c9cba3',  # Púrpura intenso
    '#66cccc',  # Verde azulado intenso
    '#ff66b3',  # Rosa intenso
    '#ffb056',  # Verde claro intenso
    '#98c1d9',  # Rojo anaranjado intenso
    '#ffafcc'   # Verde intenso
]

In [ ]:
collisions.drop(columns=['ZIP CODE', 'STREET NAME'])
collisions.dropna(inplace=True)

### Are accidents more frequent during weekdays or weekends? Is there any difference between before COVID-19 and after?

To answer the question in a pretty straightforward way, we decided that a slope chart would be a really good option since we first intend to show the difference between the average of weekdays and weekends for both years, so using a barchart would probably not be optimal. We consider that the results allow to answer the question pretty easily, however we want to explore some options that give the user more detail showing, for instance, all the days of the week.

In [ ]:
collisions['TYPE OF DAY'].unique()

In [ ]:
df = collisions.loc[:, ['YEAR', 'DAY NAME', 'TYPE OF DAY']]
df.insert(0, 'COUNT', 1)

df = df.groupby(['YEAR', 'TYPE OF DAY']).count().reset_index()
# divide count by 5 if it is a weekday
df['COUNT'] = df.apply(lambda x: x['COUNT']/5 if x['TYPE OF DAY'] == 'Weekday' else x['COUNT']/2, axis=1)


slope = alt.Chart(df).mark_line().encode(
    x=alt.X('YEAR:N', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('COUNT:Q', title='Collisions per Day'),
    color=alt.Color('TYPE OF DAY:N', legend=alt.Legend(title='Day Type'))
)

pts = alt.Chart(df).mark_point(
    filled=True,
    opacity=1
).encode(
    x=alt.X('YEAR:N', title='Year', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('COUNT:Q', title='Collisions per Day'),
    color=alt.Color('TYPE OF DAY:N',
                    scale=alt.Scale(range=['#B3E9C7', '#8367C7']),
                    legend=None)
)
alt.layer(slope, pts).properties(width=100, height=300, title='Collisions per Day Type by Year')

In [ ]:
df = collisions.groupby('YEAR')[['NUMBER OF INJURED']].sum().reset_index()

df2 = collisions.groupby('YEAR')[['COLLISION_ID']].count()
df = df.merge(df2, on='YEAR')

df['TOTAL PER 10'] = 10 * df['NUMBER OF INJURED'] / df['COLLISION_ID']
df = df.reset_index()

In [ ]:
car = ("M640 320V368C640 385.7 625.7 400 608 400H574.7C567.1 445.4 527.6 480 480 480C432.4 480 392.9 445.4 385.3 400H254.7C247.1 445.4 207.6 480 160 480C112.4 480 72.94 445.4 65.33 400H32C14.33 400 0 385.7 0 368V256C0 228.9 16.81 205.8 40.56 196.4L82.2 92.35C96.78 55.9 132.1 32 171.3 32H353.2C382.4 32 409.1 45.26 428.2 68.03L528.2 193C591.2 200.1 640 254.8 640 319.1V320zM171.3 96C158.2 96 146.5 103.1 141.6 116.1L111.3 192H224V96H171.3zM272 192H445.4L378.2 108C372.2 100.4 362.1 96 353.2 96H272V192zM525.3 400C527 394.1 528 389.6 528 384C528 357.5 506.5 336 480 336C453.5 336 432 357.5 432 384C432 389.6 432.1 394.1 434.7 400C441.3 418.6 459.1 432 480 432C500.9 432 518.7 418.6 525.3 400zM205.3 400C207 394.1 208 389.6 208 384C208 357.5 186.5 336 160 336C133.5 336 112 357.5 112 384C112 389.6 112.1 394.1 114.7 400C121.3 418.6 139.1 432 160 432C180.9 432 198.7 418.6 205.3 400z"
)

In [ ]:
data = pd.DataFrame([dict(id=i) for i in range(1, 11)])

data['color'] = ['kill/injured' if i < 6 else 'non' for i in range(1, 11)]

data['year'] = ['2018'] * 10

alt.Chart(data).transform_calculate(
    row="ceil(datum.id/10)"
).transform_calculate(
    col="datum.id - datum.row*10"
).mark_point(
    filled=True,
    size=0.04
).encode(
    alt.X("col:O", axis=None),
    alt.Y("row:O", axis=None),
    alt.ShapeValue(car),
    color=alt.Color('color:N',
                    scale=alt.Scale(domain=['kill/injured', 'non'],range=['#5603ad', '#9BA8C7']),
                    legend=None)
).properties(
    width=900,
    height=130
)

In [ ]:
data = pd.DataFrame([dict(id=i) for i in range(1, 11)])

data['color'] = ['kill/injured' if i < 10 else 'non' for i in range(1, 11)]

data['year'] = ['2020'] * 10

alt.Chart(data).transform_calculate(
    row="ceil(datum.id/10)"
).transform_calculate(
    col="datum.id - datum.row*10"
).mark_point(
    filled=True,
    size=0.04
).encode(
    alt.X("col:O", axis=None),
    alt.Y("row:O", axis=None),
    alt.ShapeValue(car),
    color=alt.Color('color:N',
                    scale=alt.Scale(domain=['kill/injured', 'non'],range=['#5603ad', '#9BA8C7']),
                    legend=None)
).properties(
    width=900,
    height=130
)

The first and simpler option to give more detail than in the previous chart is to make a barplot. With just to options it wasn't such a good idea, but if we show all the days of the week it makes much more sense. Since we want the names of the days to be legible the idea is to make an horizontal bar chart.

In [ ]:
alt.data_transformers.enable("vegafusion")

In [ ]:
df = collisions[['DAY NAME']]

alt.Chart(df).mark_bar().encode(
    x=alt.X('count():Q',
            title='Number of Collisions'),
    y=alt.Y('DAY NAME:O',
            sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            title='Day of the Week'),
    color=alt.Color('DAY NAME:O',
                    scale=alt.Scale(domain=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                                    range=['#B3E9C7', '#B3E9C7', '#B3E9C7', '#B3E9C7', '#B3E9C7', '#8367C7', '#8367C7']),
                                    legend=None)
).properties(
    width=500,
    height=200,
    title='Number of Collisions by Day of the Week'
)

As mentioned, this option is simple but effective in its task. However, we are also asked to show if there is a difference between before and after COVID-19 (that is between 2018 and 2020), so in future iterations of the visualization we should take this into account.

The next visualizaiton intends to answer the same question, but also tries to show if there is a time period of the day (morning, afternoon, night) that has more crashes than others. Nevertheless, the outcome is not really useful since the comparison between bars and inside bars is pretty diffuclt, moreover, the colors catch the attention so it's also more difficult (it takes longer since there are more variables encoded) than in the previous visualization to identify the days with more crashes. Again, it fails to show a differentiation between years so we shall look for other options.

In [ ]:
df = collisions[['DAY NAME', 'CRASH MOMENT']]

bars = alt.Chart(df).mark_bar().transform_calculate(
    order="{'Morning': 1, 'Afternoon': 2, 'Night': 3}[datum['CRASH MOMENT']]"
).encode(
    x=alt.X('count():Q',
            title='Number of Collisions',
            stack='zero'),
    y=alt.Y('DAY NAME:N',
            sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            title='Day of the Week'),
    color=alt.Color('CRASH MOMENT:N',
                    title='Moment of the Day',
                    scale=alt.Scale(domain=['Morning', 'Afternoon', 'Night'],
                                    range=['#B3E9C7', '#9BA8C7', '#8367C7'])),
    order=alt.Order('order:O')
).properties(
    width=500,
    height=200,
    title='Number of Collisions by Day of the Week'
)

text = alt.Chart(collisions[['DAY NAME', 'CRASH MOMENT']]).mark_text(
    align='right',
    color='black',
    baseline='middle'
).transform_calculate(
    order="{'Morning': 1, 'Afternoon': 2, 'Night': 3}[datum['CRASH MOMENT']]"
).encode(
   x=alt.X('count():Q',
            title='Number of Collisions',
            stack='zero'),
    y=alt.Y('DAY NAME:N',
            sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            title='Day of the Week'),
    detail=alt.Detail('CRASH MOMENT:N'),
    text=alt.Text('count():Q'),
    order=alt.Order('order:O')
)

bars + text

To deal with the differentiation between 2018 and 2020, in which the previous visualizations fail, an interesting option is to do a paired horizontal bar chart. Since there is only two classes inside every row, (theoretically) the comaprison between instances from different rows is not as difficult as it would be if there were more categories. Therefore, we consider this visualization to accomplish its objectives, but we will keep iterating to see if a better result can be found. Therefore, we consider that this visualization should accomplish its objectives and be useful t answer the questions.

In [ ]:
df = collisions[['YEAR', 'DAY NAME']]

alt.Chart(df).transform_aggregate(
    count='count()',
    groupby=['DAY NAME', 'YEAR']
).mark_bar().encode(
    x=alt.X('count:Q',
            title='Number of Collisions'),
    y=alt.Y('YEAR:O',
            axis=alt.Axis(grid=False, ticks=False, labels=False),
            title=''),
    color=alt.Color('YEAR:N',
                    scale=alt.Scale(domain=['2018', '2020'],
                                    range=['#B3E9C7', '#8367C7']),
                    legend=alt.Legend(title='Year')),
    row=alt.Row('DAY NAME:O',
                      header=alt.Header(title='Day of the Week'),
                      sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                      title='Day of the Week')
).properties(
    title='Number of Collisions by Day of the Week and Year'
)

Contrary to what we thought, the result is not as good as expected. Because of the separation per days, the comparison between days is not so easy, and this problem is accentuated for the 2020 instances which vary less and because of this and the separation the differences are more difficult to notice. Moreover, the names of the days are in vertical which makes them diffuclt to read; this problem can be solved easily but teh previously mentioned ones will still be there.

Trying to ease the comparison between instances, an option we considered useful was a small multiples of the initial barchart, one for each year. At first this may not seem a great option if we are trying to enhance the comparison, but we consider that the fact of only havig two barcharts that share indexes and are aligned makes the comparison between instances of the different charts pretty easy and, of course, the comparison between instances of the same chart is as good as for the initial plot.

In [ ]:
df = collisions[['YEAR', 'DAY NAME']]

alt.Chart(df).mark_bar().encode(
    x=alt.X('count():Q'),
    y=alt.Y('DAY NAME:O',
            title='Day of the Week',
            sort=alt.SortField(field='DAY NAME', order='ascending')),
    color=alt.Color('YEAR:O',
                    scale=alt.Scale(domain=['2018', '2020'], range=['#C2F8CB', '#8367C7']),
                    legend=None),
    column=alt.Column('YEAR:O', title='Year')
).properties(
    width=200,
    height=200,
    title='Number of Collisions by Day of the Week (2018 vs. 2020)'
)

The main flaw of this plot is that it may be "wasting some space" and it may not be the best option to see the general trend.

Since we will be showing more detail of the days of the week in the visualization for the third question, we finally decided to use the slope chart for this question. As mentioned earlier, we consider that it is really useful to answer the question posed. Moreover it is simple and does not occupy as much space as the other options. The main flaw of this plot, even if the question doesn't ask explicitly for it, is that some detail is lost when aggregating all days in weekday or weekend.

### Is there any type of vehicle more prone to participate in accidents?

In [ ]:
collisions['YEAR'] = collisions['YEAR'].astype(str)

In [ ]:
collisions["VEHICLE TYPE CODE 1"].unique()

Given that we aggregated the type of vehicles to a total of just 13 classes, the bar chart is the first option that comes to our mind.

In [ ]:
df = collisions[['VEHICLE TYPE CODE 1']]

alt.Chart(df).mark_bar().encode(
    x=alt.X('VEHICLE TYPE CODE 1:N', title='Vehicle Type', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count():Q', title='Number of Collisions')
).properties(
    width=500,
    height=200,
    title='Number of Collisions by Vehicle Type'
).configure_mark(
    color='#6D35BA'
)

The result is decent but at the same time there are some classes barely noticeable. We want to clarify that the answer of this question is impossible with the available data, since we would need the traffic percenatge or proportion of every type of vehicle to determine if there is a vehicle more prone to have crashes than others. This happens because, for example, there are much more cars than ambulances, so the number of total car crashes is much bigger than the numbe of total ambulance crashes, and without the traffic proportions we can't really say if one of the vehicles is more prone to have an accident than the others.

Having stated this, we thought that polar area charts could be a good option to make the classes with less crashes more noticeable. We are aware that the comparison between areas is more difficult, however we believe that the main (erroneous for the reasons previously explained) conclusions for the question are still clear enough and pretty understandable.

In [ ]:
alt.Chart(collisions).encode(
    alt.Theta("VEHICLE TYPE CODE 1:N",
              stack = True,
              sort=alt.EncodingSortField(field="count", op="count", order='descending')),
    alt.Radius("count()",
               scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("VEHICLE TYPE CODE 1:N",
                    sort=alt.EncodingSortField(field="count", op="count", order='descending'),
                    scale=alt.Scale(range=colores_hex),
                    legend=alt.Legend(title="Vehicle Type")),
).mark_arc(
    innerRadius=5, stroke="#fff"
)

The choice of this palette with no colors being extremely similar or much more appealing than others allows for a good idenftification of the thirteen instances. The strength of the chart is taht allows to see the groups with less total crashes, so the user can compare the smaller classes aswell as the bigger ones. However, the problem is, as explained previously, that areas are much more difficult to compare than lenghts, so although the user can say which are bigger than others (since its ordered by size), he can't really quantify how much bigger are ones from others.

To solve this problem, we determined that the solution would be to aggregate the numbers with the actual counts of crashes to each of the instances. Which resulted in the following plot:

In [ ]:
c1 = alt.Chart(collisions).encode(
    alt.Theta("VEHICLE TYPE CODE 1:N",
              stack = True,
              sort=alt.EncodingSortField(field="count", op="count", order='descending')),
    alt.Radius("count()",
               scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color=alt.Color("VEHICLE TYPE CODE 1:N",
                    sort=alt.EncodingSortField(field="count", op="count", order='descending'),
                    scale=alt.Scale(range=colores_hex),
                    legend=alt.Legend(title="Vehicle Type")),
).mark_arc(
    innerRadius=5, stroke="#fff"
)

text = c1.mark_text(
    align='center',
    baseline='middle',
    radiusOffset=20,
    fontSize=12.5,
).encode(
    text='count()'
)

alt.layer(c1 + text).properties(
    title='Number of Collisions by Vehicle Type'
)

In [ ]:
# create a new column that sums NUMBER OF PERSONS INJURED and NUMBER OF PERSONS KILLED
collisions['TOTAL INJURED/KILLED'] = collisions['NUMBER OF INJURED'] + collisions['NUMBER OF KILLED']

The last option we considered is a lollipop chart since it allows an easy comparison of several instances. The result is the following and it is similar to the one of a typical barchart. Simple and effective, but with the same problem of some types of vehicle being unnoticeable and impossible to differentiate (the ones with the lowest counts of crashes).

In [ ]:
df = collisions[['VEHICLE TYPE CODE 1', 'TOTAL INJURED/KILLED']]

df1 = df.groupby('VEHICLE TYPE CODE 1').sum("TOTAL INJURED/KILLED").reset_index()
df2 = df.groupby('VEHICLE TYPE CODE 1').count().reset_index()

df2.columns = ['VEHICLE TYPE CODE 1', 'TOTAL COLLISIONS']

df = pd.merge(df1, df2, on='VEHICLE TYPE CODE 1')


lolli = alt.Chart(df).mark_bar(
    size=3
).encode(
    x=alt.X('TOTAL COLLISIONS:Q',
            title='Total Collisions'),
    y=alt.Y('VEHICLE TYPE CODE 1:N',
            title='Vehicle Type',
            sort=alt.EncodingSortField(field="TOTAL COLLISIONS", order="descending"),
            axis=alt.Axis(labelAngle=0, grid=True)),
    color=alt.Color('VEHICLE TYPE CODE 1:N',
                    title='Vehicle Type',
                    legend=None),
)

pop = alt.Chart(df).mark_circle(
    tooltip=True,
    size=80,
    opacity=1
).encode(
    x=alt.X('TOTAL COLLISIONS:Q',
            title='Total Collisions'),
    y=alt.Y('VEHICLE TYPE CODE 1:N',
            title='Vehicle Type',
            sort=alt.EncodingSortField(
                field="TOTAL COLLISIONS",
                order="descending"),
            axis=alt.Axis(labelAngle=0, grid=True)),
    color=alt.Color('VEHICLE TYPE CODE 1:N',
                    title='Vehicle Type',
                    legend=None),
    tooltip=['VEHICLE TYPE CODE 1:N', 'TOTAL COLLISIONS:Q', 'TOTAL INJURED/KILLED:Q']
).properties(
    title='Lollipop Plot of Collisions by Vehicle Type and Contributing Factor'
)

lolli + pop

We decided that the best option in this case is the last iteration of the polar area chart. It solves the problem of less common classes being really difficult to differentiate and offers the actual count values so that the user can get an idea of how much difference is between the number of accidents of the different vehicle types. A possible problem is that for this type of chart so many instances could be difficult to interpret at first glance, but since we have chosen a proper palette, ordered the instances by value and written their values, we consider this is just a minor problem that should not affect much the aim of the visualization.

### At what time of the day are accidents more common?

In [ ]:
collisions['YEAR'] = collisions['CRASH DATE'].astype(str).str[:4]

Since we aggregated the count accidents by time of the day in hours, we consider that a line chart is a great option to represent the evolution of total car crashes per hours. Also, this type of chart allows to separate the data by years (2018, 2020) very easlily to add extra information. As we can see the result clearly shows the top hours with more crashes and allows an easy comparison between hours and years.

In [ ]:
df = collisions[['COLLISION_ID', 'CRASH TIME INTERVAL', 'YEAR']]

alt.Chart(df).mark_area(
    point=True,
    fillOpacity=0.8,
    line=True,
    tooltip=True
    # interpolate='monotone'
).transform_aggregate(
    count='count()',
    groupby=['YEAR', 'CRASH TIME INTERVAL']
).encode(
    x=alt.X('CRASH TIME INTERVAL:O', title='Hour of the Day', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('count:Q', title='Number of Collisions', stack=None),
    color=alt.Color('YEAR:N', title='Year', scale=alt.Scale(domain=['2018', '2020'], range=['lightblue', 'lightgreen'])),
    tooltip=['count:Q', 'CRASH TIME INTERVAL:O']
).properties(
    title='Number of Collisions by Hour of the Day'
)

Another option we considered is a heatmap. This time we also add the day of the week and not only the time of the day, so this visualization not only helps knowing in which time of the day there are more crashes but also allows us to answer the question that asks us to analyze if there are more crashes in weekends or in weekdays, giving more detail and context to the first question. We consider the result to be pretty useful and clear. The only issue is that this visualization does not allow to compare between years, however this is not an important problem since, as mentioned, there is a specific visualization for that task.

In [ ]:
collisions.head(1)

In [ ]:
c1 = alt.Chart(collisions[['CRASH TIME INTERVAL', 'DAY NAME', 'YEAR']]).mark_rect(
    tooltip=True
).encode(
    x=alt.X('CRASH TIME INTERVAL:N',
            title='Hour of the Day',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('DAY NAME:N',
            sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
            title='Day of the Week'),
    color=alt.Color('count():Q',
                    title='Number of Collisions',
                    scale=alt.Scale(range=['#f0fff1', '#5603ad'])),
    tooltip=['count()']
).properties(
    title='Number of Collisions by Hour of the Day and Day of the Week'
)

c1

While the line chart is a great option, we concluded that the best option for this case is the heatmap. This is mainly because it gives an extra layer of detail and this detail complements the visualization for the first question. The color palette chosen also allows people affected by color blindness to be able to draw conclusions without any difficulties.

### Are there any areas with a larger number of accidents?

This is the map of New York City diveded in boroughs, we first thought about using a dot map. However, the great amount of instances, the limited space for the visualizations and the restriction of not using interactive plots, made us change our mind after some simple tries.

In [ ]:
nyc = alt.Chart(nyc_map).mark_geoshape(
    stroke='white',
    strokeWidth=1,
    filled=True
).encode(
    color=alt.Color('name',
                    scale=alt.Scale(scheme='greys'),
                    title='Borough'),
).project(
    type='identity', reflectY=True
)
nyc

We decided to do a choropleth map, but dividing the main regions into smaller geometrical forms (hexagons), so the areas would be easier to compare. We use a sequential palette that represents the quantites aggrupated in every hexagon, so the user can see in which areas there is a larger concentration of accidents. We decided to keep the shape of the original map to make orientation easier for the user.

In [ ]:
# df = nyc_map_hex[['h3_polyfill', 'name', 'geometry']]
nyc_map_hex = nyc_map.h3.polyfill_resample(8)
nyc_map_hex = nyc_map_hex.reset_index()
nyc_map_hex = nyc_map_hex[['h3_polyfill', 'name', 'geometry']]

In [ ]:
collisions_geo = gpd.GeoDataFrame(collisions[['COLLISION_ID']],
                                  geometry=gpd.points_from_xy(collisions['LONGITUDE'], collisions['LATITUDE']))

collisions_geo.crs = nyc_map_hex.crs
df = collisions_geo.sjoin(nyc_map_hex, how='right')

tmp = df.groupby(['h3_polyfill', 'name']).count().reset_index()
tmp = tmp[['h3_polyfill', 'name', 'COLLISION_ID']]
tmp.columns = ['h3_polyfill', 'name', 'count']

df = df.merge(tmp, on=['h3_polyfill', 'name'], how='left')
df = df[['h3_polyfill', 'name', 'geometry', 'count']]
df = df.drop_duplicates(subset=['h3_polyfill', 'name'])

if not os.path.exists(f'{dir}/new-york-city-boroughs-ny_hex.geojson'):
    df.to_file(f'{dir}/new-york-city-boroughs-ny_hex.geojson', driver='GeoJSON')

In [ ]:
c1 = alt.Chart(df).mark_geoshape(
    stroke='white',
    strokeWidth=1,
    filled=True
).encode(
    color=alt.Color('count:Q',
                    # scale=alt.Scale(scheme='greys'),
                    # scale=alt.Scale(scheme='tealblues'),
                    scale=alt.Scale(range=['#B3E9C7', '#5603ad']),
                    title='Borough'),
).project(
    type='identity', reflectY=True
).properties(
    width=500,
    height=500
)

c2 = alt.Chart(nyc_map).mark_geoshape(
    stroke='#1d3557',
    strokeWidth=1,
    filled=False
).project(
    type='identity', reflectY=True
).properties(
    width=500,
    height=500
)

c1 + c2

For the final visualization, the addition of the borough labels was introduced to ease the distinction of boroughs.

In [ ]:
# get centroid of each borough
boroughs = nyc_map[['name', 'geometry']]
boroughs['centroid'] = boroughs['geometry'].centroid
boroughs = boroughs.set_geometry('centroid')
boroughs['lat'] = boroughs['centroid'].apply(lambda p: p.y)
boroughs['lon'] = boroughs['centroid'].apply(lambda p: p.x)
boroughs = boroughs.drop(['centroid', 'geometry'], axis=1)

if not os.path.exists(f'{dir}/new-york-city-boroughs-names.csv'):
    boroughs.to_csv(f'{dir}/new-york-city-boroughs-names.csv', index=False)

In [ ]:
nyc_map_hex.crs

In [ ]:
nyc_map.crs

In [ ]:
# select LATITUDE and LONGITUDE columns and convert them to a GeoDataFrame
collisions_geo = gpd.GeoDataFrame(collisions[['LATITUDE', 'LONGITUDE', 'BOROUGH', 'ZIP CODE', 'TOTAL INJURED/KILLED']], geometry=gpd.points_from_xy(collisions.LONGITUDE, collisions.LATITUDE))

collisions_geo.crs = "EPSG:4326"
collisions_geo.crs

In [ ]:
population = pd.DataFrame({
    'BOROUGH': ['BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS', 'STATEN ISLAND'],
    'POPULATION_2018': [1438000000, 2601000000, 1632000000, 2299000000, 474101],
    'POPULATION_2020': [1427000000, 2577000000, 1629000000, 2271000000, 475596],
    'CAR OWNERSHIP': [0.40, 0.44, 0.22, 0.62, 0.83]
})

population['MEAN POPULATION'] = population[['POPULATION_2018', 'POPULATION_2020']].mean(axis=1)

population

In [ ]:
total_population = population['MEAN POPULATION'].sum()

df = collisions[['BOROUGH', 'CRASH TIME INTERVAL']]
df.insert(0, 'COUNT', 1)

df = df.groupby(['BOROUGH', 'CRASH TIME INTERVAL']).count().reset_index()
df = df.merge(population[['BOROUGH', 'MEAN POPULATION', 'CAR OWNERSHIP']], on='BOROUGH', how='left')

df['NORMALIZED COUNT'] = df['COUNT'] * df['CAR OWNERSHIP']

df.describe()

Although we consider the map to be a pretty good option, we also thought that a line chart representing the normalized count of accidents along the day hours for each borough could also be a good option to answer the proposed question. We normalized the count of accidents with the information of the follwoing [article](https://edc.nyc/article/new-yorkers-and-their-cars), which specifies the percentage of people in every borough who owns a car. This also helps to give an extra view on the most common hour of the day were accidents occur.

In [ ]:
alt.Chart(df).mark_line(
    tooltip=True,
    size=2
).encode(
    x=alt.X('CRASH TIME INTERVAL:N',
            title='Crash Time Interval',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('NORMALIZED COUNT:Q',
            title='Number of Collisions normalized by Car Ownership Data'),
    color=alt.Color('BOROUGH:N',
                    scale=alt.Scale(range=['#a3ffd6', '#d69bf5', '#ff8080', '#80ff80', '#80bfff']),
                    title='Borough')
).properties(
    title='Normalized Number of Collisions by Time Interval'
)

Finally, we have decided to use both visualizations, since by combining them the user can get a detailed view with the map and a general view with the line chart, allowing the user to draw conclusions in a pretty easy way. We consider that the main problem of this choice is that we will be occupying more space than with a single chart.

### Is there a correlation between weather conditions and accidents?

In [ ]:
weather_aggregated['DATE'] = pd.to_datetime(weather_aggregated['DATE']).dt.strftime('%Y-%m-%d')
weather_aggregated.head()

In [ ]:
collisions['CRASH DATE'] = pd.to_datetime(collisions['CRASH DATE']).dt.strftime('%Y-%m-%d')

collisions_by_day = collisions[['CRASH DATE', 'COLLISION_ID']].groupby(['CRASH DATE']).count().reset_index()
collisions_by_day.head()

In [ ]:
merged_data = pd.merge(weather_aggregated, collisions_by_day, left_on='DATE', right_on='CRASH DATE')
merged_data.drop(columns=['CRASH DATE'], inplace=True)
merged_data.rename(columns={'COLLISION_ID': 'COLLISION COUNT'}, inplace=True)
merged_data.head()

In [ ]:
merged_data['BEAUFORT_SCALE'] = merged_data['BEAUFORT_SCALE'].astype('category')
merged_data['PRCP_SCALE'] = merged_data['PRCP_SCALE'].astype('category')

In [ ]:
# Categorization of TEMPERATURE in bins of 2 Celsius degrees: min temperature is 11, max temperature is 29
bin_edges = list(range(10, 30, 2))
merged_data['TEMP_SCALE'] = pd.cut(merged_data['MEAN_TEMP'], bins=bin_edges, right=False)
merged_data['CASES_COUNT'] = 1

The normalization of the number of collisions by wind scale and temperature scale was done by divinding the total count of accidents by the number of instances of each possible combination of wind-temperature.

In [ ]:
agg_data = merged_data.groupby(['BEAUFORT_SCALE', 'TEMP_SCALE']).agg({'COLLISION COUNT': 'sum', 'CASES_COUNT': 'sum'}).reset_index()

agg_data['BEAUFORT_SCALE'] = agg_data['BEAUFORT_SCALE'].astype('category')
agg_data['TEMP_SCALE'] = agg_data['TEMP_SCALE'].astype('str')

agg_data['NORMALIZED_COLLISION_COUNT'] = agg_data['COLLISION COUNT'] / agg_data['CASES_COUNT']

agg_data.head()

Given that in summer time there is nearly no rain, we decided for this first visualization to only analyze the data given the categorized average daily speed of wind and the average daily temperature. Since we want to encode two keys; wind and temperature, a value; number of crashes, a heatmap seems to be a good option.

To do so we also categorized the temperature taking into account the maximum and minimum temperatures registered. The result is correct and relatively easy to interpret, but at the same time there are values missing for some combinations, which is expectable since the climate does not usually vary much in the same season.

In [ ]:
alt.Chart(agg_data).mark_rect().encode(
    x=alt.X('TEMP_SCALE:O',
            title='Temperature Scale',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('BEAUFORT_SCALE:N',

            title='Beaufort Scale'),
    color=alt.Color('NORMALIZED_COLLISION_COUNT:Q',
                    scale=alt.Scale(range=['#f0fff1', '#5603ad']),
                    title='Collision Count')
).properties(
    width=500,
    height=300,
    title='Collision Count by Beaufort Scale and Precipitation Scale'
)

In [ ]:
# save the file if it doesn't exist
if not os.path.exists(f'{dir}/merged_data.csv'):
    merged_data.to_csv(f'{dir}/merged_data.csv', index=False)

The other option we consider to be useful is a small multiples of the scatterplots of collisions versus the three weather variables (precipitation, temperature and wind speed). This visualization is simple and clear to show if there is any correlation between the number of accidents and this meterological factors.

In [ ]:
t1 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('MEAN_TEMP:Q',
            title='Mean Temperature',
            scale=alt.Scale(domain=[10, 30])),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    title='Year')
).properties(
    title='Number of Collisions by Mean Temperature'
)

t2 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('PRCP:Q',
            title='Mean Precipitation'),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    title='Year'),
).properties(
    title='Number of Collisions by Precipitation'
)

t3 = alt.Chart(merged_data).mark_point(
    filled=True,
    size=100,
    opacity=0.5
).encode(
    x=alt.X('AWND:Q',
            scale=alt.Scale(domain=[1, 6.5]),
            title='Mean Wind Speed'),
    y=alt.Y('COLLISION COUNT:Q',
            title='Number of Collisions'),
    color=alt.Color('year(DATE):N',
                    scale=alt.Scale(range=['#A7C9C7', '#8367C7']),
                    title='Year'),
).properties(
    title='Number of Collisions by Wind Speed'
)

t1 | t2 | t3

Since we are already using a heatmap for another question, we decided to use the small multiples for the scatterplots. As mentioned, this option is effective and really allows the user to see if there is any clear correlation. The main disadvantage of this plot is that it may occupy more space than other options.

### Extra Visualizations

The first extra visualization we have chosen to create is a isotype plot, showing the proportion of injured people in 2018 and 2020 with car shapes representing this information. We also calculate some KPI's we thought interesting to show in the final visualization, like the number of deaths in each year, the total amount of collisions each year, etc.

In [ ]:
df = collisions.groupby('YEAR')[['NUMBER OF INJURED', 'NUMBER OF KILLED']].sum().reset_index()

df2 = collisions.groupby('YEAR')[['COLLISION_ID']].count()
df = df.merge(df2, on='YEAR')

df['TOTAL PER 10'] = 10 * df['NUMBER OF INJURED'] / df['COLLISION_ID']
df

In [ ]:
car = ("M640 320V368C640 385.7 625.7 400 608 400H574.7C567.1 445.4 527.6 480 480 480C432.4 480 392.9 445.4 385.3 400H254.7C247.1 445.4 207.6 480 160 480C112.4 480 72.94 445.4 65.33 400H32C14.33 400 0 385.7 0 368V256C0 228.9 16.81 205.8 40.56 196.4L82.2 92.35C96.78 55.9 132.1 32 171.3 32H353.2C382.4 32 409.1 45.26 428.2 68.03L528.2 193C591.2 200.1 640 254.8 640 319.1V320zM171.3 96C158.2 96 146.5 103.1 141.6 116.1L111.3 192H224V96H171.3zM272 192H445.4L378.2 108C372.2 100.4 362.1 96 353.2 96H272V192zM525.3 400C527 394.1 528 389.6 528 384C528 357.5 506.5 336 480 336C453.5 336 432 357.5 432 384C432 389.6 432.1 394.1 434.7 400C441.3 418.6 459.1 432 480 432C500.9 432 518.7 418.6 525.3 400zM205.3 400C207 394.1 208 389.6 208 384C208 357.5 186.5 336 160 336C133.5 336 112 357.5 112 384C112 389.6 112.1 394.1 114.7 400C121.3 418.6 139.1 432 160 432C180.9 432 198.7 418.6 205.3 400z"
)

idx = 6
# idx = 10

data = pd.DataFrame([dict(id=i) for i in range(1, 11)])
data['color'] = ['kill/injured' if i <= idx else 'non' for i in range(1, 11)]
data['year'] = ['2018'] * 10

alt.Chart(data).transform_calculate(
    row="ceil(datum.id/10)"
).transform_calculate(
    col="datum.id - datum.row*10"
).mark_point(
    filled=True,
    size=0.03,
    tooltip=False
).encode(
    alt.X("col:O", axis=None),
    alt.Y("row:O", axis=None),
    alt.ShapeValue(car),
    color=alt.Color('color:N',
                    scale=alt.Scale(domain=['kill/injured', 'non'],range=['#5603ad', '#9BA8C7']),
                    legend=None)
).properties(
    width=800,
    title=f'In "year", in average, around {idx} out of 10 Collisions involve injured'
).configure_axis(
    grid=False,
    domain=False,
    ticks=False
).configure_view(
    strokeWidth=0
)

The second extra visualization consists of a bar plot with the top 5 factors that contributed to accidents, with the number of accidents caused by every factor. Since a lot of instances were undefined in this attribute, we removed the factor 'Undefined' that doesn't apport any information.

In [ ]:
df = collisions[['CONTRIBUTING FACTOR VEHICLE 1', 'COLLISION_ID']]

df = df.groupby('CONTRIBUTING FACTOR VEHICLE 1').count().reset_index()

df = df.sort_values(by='COLLISION_ID', ascending=False)

df = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

df.columns = ['CONTRIBUTING FACTOR VEHICLE 1', 'COUNT']
df = df.head(5)

alt.Chart(df).mark_bar(
    tooltip=True
).encode(
    x=alt.X('CONTRIBUTING FACTOR VEHICLE 1:N',
            title='Contributing Factor',
            sort=alt.EncodingSortField(field="COUNT", order="descending"),
            axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('COUNT:Q',
            title='Number of Collisions'),
    tooltip=['CONTRIBUTING FACTOR VEHICLE 1:N', 'COUNT:Q']
).properties(
    title='Number of Collisions by Contributing Factor'
).configure_mark(
    color='#ff8080'
)

In [ ]:
df = collisions.groupby('YEAR')[['NUMBER OF INJURED', 'NUMBER OF KILLED']].sum().reset_index()

df2 = collisions.groupby('YEAR')[['COLLISION_ID']].count()
df = df.merge(df2, on='YEAR')

df['TOTAL PER 10'] = 10 * df['NUMBER OF INJURED'] / df['COLLISION_ID']

df

## Answers to the Questions

### Question 1: Are accidents more frequent during weekdays or weekends? Is there any difference between before COVID-19 and after?

As we can see in the third row of plots, the slope chart shows that in both years accidents were more common during weekdays than during weekends. This is can also be seen in detail in the adjacent heatmap, where, in average, there were more crashes during weekdays than during weekends. However, the slope chart also shows that in 2020 the difference is reduced considerably, porbably due to the massive adoption of teleworking after COVID-19.

### Question 2: Is there any type of vehicle more prone to participate in accidents?

As explained in the generation of the plots, this can't really be answered with the data given. Nevertheless, the polar area chart in the top left corner, just under the title, clearly shows that the vehicles with a higher amount of accidents are cars and vans, considering them as very general categories. With a massive difference, they are followed by trucks and taxis (a very common vehicle in NYC).

### Question 3: At what time of the day are accidents more common?

In the third row, looking again at the previously mentioned heatmap, we observe that the highest number of crashes occur during the most common work schedules: between 8 AM and 6 PM. The peak of crashes takes place at 4 PM. It is interesting to see than in weekends, although the previous analysis sort of maintains, the number of crashes in the early past midnight hours is higher than in weekdays, probably because during weekends people tend to hang out until late hours of the night.

### Question 4: Are there any areas with a larger number of accidents?

Looking at the line chart next to the polar area chart, we observe that for any time of the day, there are much more accidents in Queens, followed pretty closely by Brooklyn. However, if we look to small zones with a major concentration of accidents in the map under the previous plot, we see that Manhattan has a larger concentration of accidents in a smaller area.

### Question 5: Is there a correlation between weather conditions and accidents?

The three scatterplots under the heatmap show that there is not any type of correlation between the meteorological conditions considered and the number of crashes. Neither temperature, nor precipitation, nor wind speed seem to show any type of trend with the amount of crashes. It is also true, that since we are only treating data from summer time, this analysis can't be done as well as it should, since meterorological conditions tend to vary very slightly during a season. This remains the same for both 2018 and 2020.


### Extra Visualization 1

The visualization under the just analyzed scatterplots shows the proportion of injured people of the total number of crashes for both years, also showing the deaths and the total crashes. It clearly shows that although the number of crashes reduced considerably after COVID-19, the dangerousness of them increased considerably, since the proportion of injured people is much bigger, and the number of deaths increased a lot proportionally.


### Extra Visualization 2

Next to the map, we can observe a barplot showcasing the top 5 causes for accidents in the summer months of 2018 and 2020 from the registered data.
The main factor with a big difference is "Driver Inattention/Distraction", follwed then by "Following Too Closely", making these two the main reasons of accidents in NYC during the summers of 2018 and 2020.